In [ ]:
# Version 1
# CNN-LSTM implementation to predict air quality index
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
df = pd.read_csv('data/ml-ready/daily-AQI-socio-economic-ml-ready.csv')
print(df)
df.shape

In [ ]:
df.keys()

In [ ]:
df.describe()

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt

# Preprocess the data
features = df.drop(columns=['Date Local','AQI']).values

target = df['AQI'].values

# Normalize the features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Create sequences for LSTM
def create_sequences(data, target, sequence_length=5):
    sequences = []
    labels = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i + sequence_length])
        labels.append(target[i + sequence_length])
    return np.array(sequences), np.array(labels)

sequence_length = 15
X, y = create_sequences(features_scaled, target, sequence_length)

# Convert to PyTorch tensors
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create DataLoader for batching
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the CNN-LSTM model
class CNN_LSTM(nn.Module):
    def __init__(self):
        super(CNN_LSTM, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=X_train.shape[2], out_channels=64, kernel_size=2)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=2)
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.lstm1 = nn.LSTM(input_size=128, hidden_size=100, batch_first=True)
        self.lstm2 = nn.LSTM(input_size=100, hidden_size=50, batch_first=True)
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(50, 25)
        self.fc2 = nn.Linear(25, 1)

    def forward(self, x):
        x = x.permute(0, 2, 1)  # Change shape to (batch_size, num_features, sequence_length)
        x = self.conv1(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.pool(x)
        x = x.permute(0, 2, 1)  # Change shape back to (batch_size, sequence_length, num_features)
        x, (hn, cn) = self.lstm1(x)
        x, (hn, cn) = self.lstm2(x)
        x = self.dropout(x[:, -1, :])
        x = self.fc1(x)
        x = self.fc2(x)
        return x

model = CNN_LSTM()

# Define loss and optimizer
criterion = nn.MSELoss() #1/n(y_hat - y)squared
# criterion = nn.SmoothL1Loss()  # Huber loss
optimizer = optim.Adam(model.parameters(), lr=0.0001)

train_losses = []
val_losses = []

# Training loop
num_epochs = 250
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs.squeeze(), batch_y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    # train_loss /= len(train_loader)
    train_losses.append(train_loss)

    # Validation loop
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            outputs = model(batch_X)
            loss = criterion(outputs.squeeze(), batch_y)
            val_loss += loss.item()

    # val_loss /= len(test_loader)
    val_losses.append(val_loss)

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss/len(train_loader):.4f}, Val Loss: {val_loss/len(test_loader):.4f}')

# Evaluate the model
model.eval()
test_loss = 0
with torch.no_grad():
    for batch_X, batch_y in test_loader:
        outputs = model(batch_X)
        loss = criterion(outputs.squeeze(), batch_y)
        test_loss += loss.item()
print(f'Test Loss: {test_loss/len(test_loader):.4f}')

# Predict future AQI values
model.eval()
with torch.no_grad():
    predictions = model(X_test).squeeze().numpy()
print(predictions)

In [ ]:
print(predictions)

In [ ]:
print(y_test)

In [ ]:
# Save the trained model
torch.save(model.state_dict(), 'deeper_cnn_lstm_attention_model.pth')

In [ ]:
train_losses_scaled = [x/len(train_loader) for x in train_losses]
val_losses_scaled = [x/len(train_loader) for x in val_losses]

# Plot the training and validation loss
epochs = range(1, len(train_losses) + 1)
plt.figure(figsize=(10, 5))
plt.plot(epochs, train_losses_scaled, label='Train Loss')
plt.plot(epochs, val_losses_scaled, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
true_values = np.array(y_test)

# Calculate performance metrics
mse = mean_squared_error(true_values, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(true_values, predictions)
r2 = r2_score(true_values, predictions)

# Define a tolerance level for custom accuracy
tolerance = 0.3  # 30% tolerance
accuracy = np.mean(np.abs(predictions - true_values) / true_values < tolerance) * 100

print(f'MSE: {mse:.4f}')
print(f'RMSE: {rmse:.4f}')
print(f'MAE: {mae:.4f}')
print(f'R²: {r2:.4f}')
print(f'Accuracy: {accuracy:.2f}%')